In [57]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shuba\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
df=pd.read_csv("zomato_dataset\zomato.csv")
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


DATA FILTERING

In [59]:
user_budget=float(input("Enter you budget (for two person): "))
# user_budget=600 # for the time being
#user_lat=float(input("Enter you location latitiude: "))
#user_long=float(input("Enter you location longitude: "))
print("Cuisines: North Indian, South Indian, Mughlai, Chinese, Cafe, Rajasthani, Italian, Continental, Thai ,etc.")
user_cuisine=input("Enter cuisine type: ")
# user_cuisine='Chinese' 



Enter you budget (for two person): 400
Cuisines: North Indian, South Indian, Mughlai, Chinese, Cafe, Rajasthani, Italian, Continental, Thai ,etc.
Enter cuisine type: North Indian


DATA CLEANING

In [60]:
# #Deleting Unnnecessary Columns
df_filtered=df.drop(['url','dish_liked','phone'],axis=1) 
#Removing the Duplicates
df_filtered.duplicated().sum()
df_filtered.drop_duplicates(inplace=True)

#Remove the NaN values from the dataset
df_filtered.isnull().sum()
df_filtered.dropna(how='any',inplace=True)

#Changing the column names
df_filtered = df_filtered.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

# #Some Transformations 
df_filtered['cost'] = df_filtered['cost'].astype(str) #Changing the cost to string
df_filtered['cost'] = df_filtered['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
df_filtered['cost'] = df_filtered['cost'].astype(float)
#Removing '/5' from Rates
df_filtered = df_filtered.loc[df_filtered.rate !='NEW']
df_filtered = df_filtered.loc[df_filtered.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
df_filtered.rate = df_filtered.rate.apply(remove_slash).str.strip().astype('float')

# Adjust the column names
df_filtered.name = df_filtered.name.apply(lambda x:x.title())
df_filtered.online_order.replace(('Yes','No'),(True, False),inplace=True)
df_filtered.book_table.replace(('Yes','No'),(True, False),inplace=True)

df_filtered['cuisines']=df_filtered['cuisines'].astype(str)
# df_filtered['cuisines']=df_filtered['cuisines'].apply(lambda x: x.split(', '))


In [61]:
# Filter all rows for which the   user_budget-200<=cost=<user_budget+200 and user_cuisine is present

user_budget=float(user_budget)
df_filtered.drop(df_filtered[df_filtered['cost']  <= user_budget-200].index, inplace = True)
df_filtered.drop(df_filtered[df_filtered['cost']  >= user_budget+200].index, inplace = True)
df_new=df_filtered.loc[df_filtered['cuisines'].str.contains(user_cuisine)]
df_new.head()


,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
20,"146, 50 ft Road, 3rd Phase, 3rd Stage, Near Ka...",The Vintage Cafe,True,False,3.0,62,Banashankari,Cafe,"Cafe, French, North Indian",400.0,"[('Rated 2.0', ""RATED\n This is a tiny cafÃ\...",[],Cafes,Banashankari
29,"112, Bull Temple Road,next to more Basavanagud...",Srinathji'S Cafe,False,False,3.8,148,Basavanagudi,"Casual Dining, Cafe","North Indian, Cafe, Chinese, Fast Food",550.0,"[('Rated 5.0', 'RATED\n No words for the pane...",[],Cafes,Banashankari
31,"2/1, 7th Main, Dwarakangar, Hosakeregalli, Ban...",Foodiction,True,False,2.8,506,Banashankari,Quick Bites,"North Indian, Fast Food, Chinese, Burger",500.0,"[('Rated 1.0', ""RATED\n Worst restaurant ever...",[],Delivery,Banashankari
34,"80, BDA Complex, 2nd Stage, Banashankari, Bang...",Faasos,True,False,4.2,415,Banashankari,Quick Bites,"North Indian, Biryani, Fast Food",500.0,"[('Rated 3.0', 'RATED\n Not worth for the mon...","['Chole Kulcha Meal', 'Upvas Aloo Paratha With...",Delivery,Banashankari


TEXT PREPROCESSING STEPS

In [62]:
## Lower Casing
df_new["reviews_list"] = df_new["reviews_list"].str.lower()

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df_new["reviews_list"] = df_new["reviews_list"].apply(lambda text: remove_punctuation(text))

## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df_new["reviews_list"] = df_new["reviews_list"].apply(lambda text: remove_stopwords(text))

df_new[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
27437,rated 20 ratedn good ordered garlic chicken ch...,"North Indian, Chinese"
15919,rated 10 ratedn worst food proper service prop...,"South Indian, North Indian, Chinese, Street Food"
4805,rated 30 ratedn packing goodit like eating tra...,"South Indian, North Indian"
25933,rated 20 ratedn reviews great zomato tried she...,"North Indian, South Indian"
11323,,"South Indian, North Indian"


In [63]:


# RESTAURANT NAMES:
restaurant_names = list(df_new['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]
    
df_new=df_new.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)
import pandas


df_fil=df_new[df_new.rate == df_new.rate.max()]
res_name=df_fil.iloc[0]['name']

TF-IDF vectorization

In [64]:
df_new.set_index('name', inplace=True)
indices = pd.Series(df_new.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_new['reviews_list'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
df_percent=df_new
df_random=df_new

Final Function to list down the restaurant based on cosine similarity  (Collaborative Filtering)

In [65]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 100 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:100].index)
    
    # Names of the top 100 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])

    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'rate', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','rate', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','rate', 'cost'], keep=False)
    df_new = df_new.sort_values(by='rate', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS HAVING %s CUISINE WITH SIMILAR REVIEWS: ' % (str(len(df_new)),user_cuisine))
    
    return df_new
recommend(res_name)

TOP 10 RESTAURANTS HAVING North Indian CUISINE WITH SIMILAR REVIEWS: 


,cuisines,rate,cost
Eat.Fit,"Healthy Food, North Indian, Biryani, Continent...",4.4,500.0
Nu.Tree,"North Indian, Healthy Food, Beverages",4.3,400.0
Eat.Fit,"Healthy Food, North Indian, Biryani, Continent...",4.1,300.0
Eat.Fit,"Healthy Food, North Indian, Biryani",4.1,300.0
Madeena Hotel,"North Indian, Mughlai, Biryani",4.0,400.0
Insomniac'S Delight,"Fast Food, North Indian",4.0,300.0
Shanthi Sagar,"South Indian, North Indian, Chinese, Juices",4.0,250.0
Madeena Hotel,"North Indian, Mughlai, Biryani",3.9,400.0
Altaf'S Chillies Restaurant,"North Indian, Chinese",3.9,500.0
Altaf'S Chillies Restaurant,"North Indian, Chinese",3.8,500.0


Other Recommender Systems for Comparisons

1. Random Recommender System

In [66]:
df_random = df_random.drop_duplicates(subset=['cuisines','rate', 'cost'], keep=False)
df_popular=df_random
df_random[['cuisines','rate','cost']].sample(10)

,cuisines,rate,cost
name,,,
Savi Sagar,"South Indian, North Indian, Chinese, Street Fo...",3.4,400.0
Kitchens@Whitefield,"Biryani, Kerala, Mughlai, Street Food, North I...",3.4,400.0
Pavana Veg Restaurant,"North Indian, South Indian, Chinese, Street Food",3.1,300.0
Sri Ruchi Restuarant,"Andhra, South Indian, North Indian",3.8,450.0
Lucky'S Biriyani Point,"Biryani, South Indian, North Indian, Fast Food",3.5,400.0
Dev'S Gugababa,"Bengali, North Indian, Chinese",3.1,500.0
Marhaba,"Arabian, Mediterranean, North Indian",3.6,500.0
Sri Udupi Park,"South Indian, North Indian, Chinese, Beverages",3.8,300.0
Ftb Restaurant,"North Indian, Bengali, Chinese, Seafood",3.6,500.0


2. Popularity Based Recommender System

In [67]:
df_popular = df_popular.sort_values(by='rate', ascending=False)
df_popular[['cuisines','rate','cost']].head(10)

,cuisines,rate,cost
name,,,
Mooch Marod,North Indian,4.5,350.0
Bitebox,"North Indian, Chinese, Fast Food, Beverages, D...",4.4,300.0
Siddiqu Kabab Center,"North Indian, Rolls",4.4,250.0
All Aromas,"North Indian, Biryani",4.3,400.0
Parrattha Ssinghh,North Indian,4.2,250.0
Nu.Tree,"North Indian, Healthy Food, Beverages",4.2,400.0
The Light House Restaurant,"North Indian, Chinese, Seafood",4.2,500.0
Purani Dilli By Anand Sweets,"North Indian, Fast Food",4.2,500.0
Fishland,"Seafood, North Indian, Chinese",4.2,500.0
